In [1]:
import numpy as np
import theano
import theano.tensor as T

import timeit

In [2]:
def load_data(dataset = 'gibbs-sample.dat.npy', borrow = True):
    """
    Loads the dataset.
    """
    data = np.load(dataset)
    dataset = theano.shared(np.asarray(data, dtype = theano.config.floatX), borrow = borrow)
        
    return dataset   

In [ ]:
class mpf(object):
    """
    Minimum probability flow
    """
    
    def __init__(self, input = None, n = 16, W = none, b = none):
        """
        Initialize mpf class
        """
        self.n = n
        
        U = np.random.rand((n,n))
        
        
        if not W:
            initial_W = np.asarray(np.fill_diagonal(0.5 * (U + U.T), 0), dtype = theano.config.floatX)
            W = theano.shared(value = initial_W, name = 'W', borrow = True)
        
        if not b:
            b = theano.shared(np.zeros(n, dtype = theano.config.floatX), name = 'b', borrow = True)
            
            
        self.W = W
        self.b = b
        
        if input is None:
            self.x = T.dmatrix(name = 'input')
        else:
            self.x = input
    
    
    def          
            
                

In [3]:
A = np.arange(9).reshape(3,3)
np.fill_diagonal(A, 0)

In [4]:
print (A)

[[0 1 2]
 [3 0 5]
 [6 7 0]]
